## Objective : We will measure performance of final model .

In [6]:
#!pip install  scikit-plot

In [7]:
#import libraries here; add more as necessary.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import sidetable
import warnings
import joblib
warnings.filterwarnings("ignore")

#Models
from xgboost import XGBClassifier


from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from pandas import ExcelWriter
from openpyxl import load_workbook

%load_ext autoreload
%autoreload 2

## Load Data

In [8]:
X_train = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_X_train_shoppersData.csv')
y_train = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_y_train.csv')

In [9]:
X_test = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_X_test_shoppersData.csv')
y_test = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_y_test.csv')

In [10]:
X_val = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_X_val_shoppersData.csv')
y_val = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_y_val.csv')

In [11]:
import Functions_File as my_ff

Functions_File is successfully imported.


# Final Model

In [12]:
print(X_train.shape ,y_train.shape)

(7398, 68) (7398, 1)


In [13]:
print(X_test.shape ,y_test.shape)

(2466, 68) (2466, 1)


In [14]:
print(X_val.shape ,y_val.shape)

(2466, 68) (2466, 1)


## Before Sampling

In [15]:
y_train.value_counts()

Revenue
0          6271
1          1127
dtype: int64

In [16]:
#USing SMOTE for oversampling of Minority class
smt = SMOTE(sampling_strategy = 1)
X_train, y_train = smt.fit_sample(X_train, y_train)

## After Sampling

In [17]:
y_train.value_counts()

Revenue
1          6271
0          6271
dtype: int64

## XGBoost

In [18]:
#model =  XGBClassifier()

In [19]:
model =  XGBClassifier(colsample_bytree=0.7, gamma=0.0,
                               learning_rate=0.05, max_depth=4,
                               min_child_weight=3, random_state=50)

In [20]:
model.fit(X_train,y_train)

XGBClassifier(colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=4,
              min_child_weight=3, random_state=50)

## Prediction on test set.  

In [21]:
y_pred_test = model.predict(X_test)
print("Accuracy:" , accuracy_score(y_test,y_pred_test)*100)

Accuracy: 88.88888888888889


In [22]:
y_pred_test_prob=model.predict_proba(X_test)

In [23]:
confusion_matrix(y_test, y_pred_test)

array([[1910,  174],
       [ 100,  282]], dtype=int64)

In [24]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred_test,  average='binary')

0.7382198952879581

In [25]:
#Compute precision, recall, F-measure and support

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.95      0.92      0.93      2084
           1       0.62      0.74      0.67       382

    accuracy                           0.89      2466
   macro avg       0.78      0.83      0.80      2466
weighted avg       0.90      0.89      0.89      2466



##  Prediction on validation set 

In [26]:
y_pred_val = model.predict(X_val)
print("Accuracy:" , accuracy_score(y_val,y_pred_val)*100)

Accuracy: 89.05109489051095


In [27]:
recall_score_val=recall_score(y_val,y_pred_val,  average='binary')
recall_score_val

0.7418546365914787

In [28]:
y_pred_val_proba =model.predict_proba(X_val)

In [29]:
#
X_val_probs = model.predict_proba(X_val)[:,1]
#%% Confusion metrics - validation set
confusion_matrix(y_val, y_pred_val)

array([[1900,  167],
       [ 103,  296]], dtype=int64)

In [30]:
#%% Compute precision, recall, F-measure and support

from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred_val))

              precision    recall  f1-score   support

           0       0.95      0.92      0.93      2067
           1       0.64      0.74      0.69       399

    accuracy                           0.89      2466
   macro avg       0.79      0.83      0.81      2466
weighted avg       0.90      0.89      0.89      2466



## Feature Importance

In [31]:
xgb_feat_imp_cover = model.get_booster().get_score(importance_type='cover')
xgb_feat_imp_gain = model.get_booster().get_score(importance_type='gain')
xgb_feat_imp_weight = model.get_booster().get_score(importance_type='weight')

In [32]:
full_feat_imp = pd.DataFrame( [ xgb_feat_imp_gain , xgb_feat_imp_cover, xgb_feat_imp_weight]).T
full_feat_imp.columns = [ 'gain' ,'cover' ,  'weight']

In [33]:
#Sort as per 'gain'
full_feat_imp.sort_values("gain", axis = 0, ascending = False , inplace = True)

In [34]:
full_feat_imp=full_feat_imp.reset_index().rename(columns = {'index' : 'features'})
full_feat_imp

,features,gain,cover,weight
0,PageValues,645.799862,981.605031,123.0
1,OperatingSystems_3,82.181446,851.396606,30.0
2,Month_May,79.647789,1022.948113,77.0
3,Month_Mar,70.896014,810.454009,62.0
4,TrafficType_3,70.302130,1159.770071,21.0
5,Month_Dec,68.885959,766.131270,60.0
6,OperatingSystems_2,65.067324,709.659998,21.0
7,OperatingSystems_4,59.623154,726.454224,1.0
8,VisitorType_Returning_Visitor,48.962885,686.479837,81.0
9,Region_3,46.178042,871.240275,42.0


## Decile

In [35]:
#Decile is implemented on Validation dataset here ,implement it  on scoring dataset in real world scenario.
#Just need to add Unique ID in Final Output.

In [38]:
def decile_func(X):
    if      (X >= 0 and X <.10) : return '10'               
    elif          (X >=.10 and X <.20): return '9'                   
    elif         (X>=.20 and X <.30): return '8'                   
    elif         (X>=.30 and X <.40): return '7'                    
    elif         (X>=.40 and X <.50): return '6'                   
    elif         (X>=.50 and X <.60): return '5'                    
    elif         (X>=.60 and X <.70): return '4'                   
    elif         (X>=.70 and X <.80): return '3'                   
    elif         (X>=.80 and X <.90): return '2'                   
    elif         (X>=.90 and X <=1): return '1'  
    else : return 'NA'


In [50]:
y_pred_val_proba[:,0]

array([0.8495533 , 0.9473635 , 0.09260315, ..., 0.8689887 , 0.8065908 ,
       0.9602581 ], dtype=float32)

In [56]:
df_proba = pd.DataFrame(y_pred_val_proba[:,0] , columns = ['Probability'])

In [63]:
df_proba.tail()


,Probability,Decile
2461,0.980810,1
2462,0.905153,1
2463,0.868989,2
2464,0.806591,2
2465,0.960258,1


In [59]:
#Apply Decile Function
df_proba['Decile'] =  df_proba['Probability'].apply(decile_func)

In [60]:
df_proba

,Probability,Decile
0,0.849553,2
1,0.947363,1
2,0.092603,10
3,0.756996,3
4,0.815701,2
...,...,...
2461,0.980810,1
2462,0.905153,1
2463,0.868989,2
2464,0.806591,2


In [62]:
df_proba['Decile'].value_counts()

1     1233
2      377
3      221
9      136
4      102
8      100
10      89
7       84
5       70
6       54
Name: Decile, dtype: int64

In [66]:
df_proba['Decile'].value_counts(normalize=True).mul(100).round(2).astype('str') + '%'

1      50.0%
2     15.29%
3      8.96%
9      5.52%
4      4.14%
8      4.06%
10     3.61%
7      3.41%
5      2.84%
6      2.19%
Name: Decile, dtype: object